In [3]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/minsir/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "minsirx.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()

import imaplib
import email
import os
import pandas as pd
from datetime import datetime
import csv
from pandas import DataFrame

import imaplib
import email
from email.header import decode_header
import os
from bs4 import BeautifulSoup
import re
from apps.email_manager.models import EmailData, EmailAttachment
from django.core.files.base import ContentFile

In [4]:


# Parent folder to save attachments (if needed for local backup)
parent_folder = "Files"

def clean_text(text):
    text = text.strip()
    text = re.sub(r'\n+', '\n', text)
    return text

def extract_main_body(email_body):
    main_body = re.split(r'From:\s', email_body, maxsplit=1)[0]
    return main_body

def html_to_text(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    for br in soup.find_all("br"):
        br.replace_with("\n")
    for p in soup.find_all("p"):
        p.insert_after("\n\n")
    for div in soup.find_all("div"):
        div.insert_after("\n")
    text = soup.get_text(separator="\n")
    return text

def decode_payload(payload, charsets):
    for charset in charsets:
        try:
            return payload.decode(charset)
        except (UnicodeDecodeError, LookupError):
            continue
    return payload.decode('latin1', errors='ignore')

def download_attachments_with_filename(email_id, email_password):
    mail = imaplib.IMAP4_SSL("outlook.office365.com", 993)
    mail.login(email_id, email_password)
    mail.select("Inbox_1")

    since_date = datetime(2023, 4, 1)
    since_date_formatted = since_date.strftime('%d-%b-%Y')
    result, data = mail.search(None, '(SINCE {date})'.format(date=since_date_formatted))
    email_nums = data[0].split()

    for num in email_nums:
        result, data = mail.fetch(num, '(RFC822)')
        raw_email = data[0][1]
        if isinstance(raw_email, int):
            continue
        msg = email.message_from_bytes(raw_email)

        encoded_subject = msg.get("Subject")
        decoded_subject, encoding = decode_header(encoded_subject)[0]
        if encoding is not None:
            decoded_subject = decoded_subject.decode(encoding)

        from_field = msg.get("From")
        decoded_from, encoding = decode_header(from_field)[0]
        if encoding is not None:
            decoded_from = decoded_from.decode(encoding)

        to_field = msg.get("To")
        decoded_to, encoding = decode_header(to_field)[0]
        if encoding is not None:
            decoded_to = decoded_to.decode(encoding)

        email_body = ""
        for part in msg.walk():
            if part.get_content_type() == "text/plain":
                charset = part.get_content_charset() or 'utf-8'
                email_body = decode_payload(part.get_payload(decode=True), [charset])
            elif part.get_content_type() == "text/html":
                charset = part.get_content_charset() or 'utf-8'
                html_content = decode_payload(part.get_payload(decode=True), [charset])
                email_body = html_to_text(html_content)

        email_body = clean_text(email_body)
        email_body = extract_main_body(email_body)

        date_str = msg.get("Date")
        splitted_1 = date_str.split("+")
        sub_date = splitted_1[0].strip()
        splitted_2 = sub_date.split("-")
        fnal_date = splitted_2[0].strip()
        fnal_date = fnal_date.replace("GMT", "").strip()
        frmts = ["%a, %d %b %Y %H:%M:%S", "%d %b %Y %H:%M:%S"]
        is_dt_made = False
        for a_dt_frmt in frmts:
            try:
                date_obj = datetime.strptime(fnal_date, a_dt_frmt)
                is_dt_made = True
                break
            except ValueError:
                pass
        if not is_dt_made:
            raise ValueError(f"unable to format date: {fnal_date}")

        email_data = EmailData(
            from_email=decoded_from,
            to_emails=[decoded_to],
            subject=decoded_subject,
            body=email_body
        )
        email_data.save()

        if msg.get_content_maintype() == 'multipart':
            for part in msg.walk():
                if part.get_content_maintype() == 'multipart':
                    continue
                if part.get('Content-Disposition') is None:
                    continue
                filename = part.get_filename()
                if filename:
                    attachment_content = part.get_payload(decode=True)
                    email_attachment = EmailAttachment(
                        email=email_data,
                        name=filename
                    )
                    email_attachment.file.save(filename, ContentFile(attachment_content))
                    email_attachment.save()

    mail.logout()

# Example usage
email_id = "mirbilal199x@outlook.com"
email_password = "APPollo1947"
download_attachments_with_filename(email_id, email_password)
